In [1]:
import arcpy
from datetime import datetime
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
# path output folder
outputs = '.\\Outputs'

## Stack Units

In [3]:
d = pd.read_csv(r'.\2020-Davis\Outputs\davis_housing_unit_inventory.csv')
s = pd.read_csv(r'.\2020-SaltLake\Outputs\salt_lake_housing_unit_inventory.csv')
w = pd.read_csv(r'.\2020-Weber\Outputs\weber_housing_unit_inventory.csv')

C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# stack em
wfrc_units = pd.concat([d,w,s])

# generate unique building id
wfrc_units['unit_id'] = wfrc_units.index + 1

# Export
wfrc_units.to_csv(os.path.join(outputs, 'wfrc_housing_unit_inventory.csv'), index=False)

In [5]:
del d
del s
del w
del wfrc_units

## Stack Units (w/ Shape)

In [6]:
ds = pd.DataFrame.spatial.from_featureclass(r'.\2020-Davis\Outputs\classes_HUI.gdb\_04_davis_housing_unit_inventory')
ss = pd.DataFrame.spatial.from_featureclass(r'.\2020-SaltLake\Outputs\classes_HUI.gdb\_04_salt_lake_housing_unit_inventory')
ws = pd.DataFrame.spatial.from_featureclass(r'.\2020-Weber\Outputs\classes_HUI.gdb\_04_weber_housing_unit_inventory')

In [7]:
# stack em
wfrc_units_shape = pd.concat([ds,ws,ss])

# generate unique building id
wfrc_units_shape['unit_id'] = wfrc_units_shape.index + 1

# create output gdb
date = datetime.today().strftime('%Y%m%d')
gdb = os.path.join(outputs, "wfrc_housing_unit_inventory_{}.gdb".format(date))
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "wfrc_housing_unit_inventory_{}.gdb".format(date))
    
wfrc_units_shape.spatial.to_featureclass(location=os.path.join(gdb, 'wfrc_housing_unit_inventory'),sanitize_columns=False)

'E:\\Projects\\Housing-Unit-Inventory\\Outputs\\wfrc_housing_unit_inventory_20210903.gdb\\wfrc_housing_unit_inventory'

In [8]:
# convert to points
arcpy.FeatureToPoint_management(os.path.join(gdb, 'wfrc_housing_unit_inventory'), 
                                os.path.join(gdb, 'wfrc_housing_unit_inventory_pts'), 
                                "INSIDE")

<Result '.\\Outputs\\wfrc_housing_unit_inventory_20210903.gdb\\wfrc_housing_unit_inventory_pts'>

In [9]:
del ds
del ss
del ws
del wfrc_units_shape